In [1]:
!pip install lxml

     |████████████████████████████████| 5.8MB 25.5MB/s eta 0:00:01


In [3]:
import requests
import lxml.html as lh
import pandas as pd

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [5]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [6]:
tr_elements = doc.xpath('//tr')
col=[]
i=0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postcode"
2:"Borough"
3:"Neighbourhood
"


#### Creating Pandas DataFrame

Each header is appended to a tuple along with an empty list.

In [8]:
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    if len(T)!=3:
        break 
    i=0 
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:        
            try:
                data=int(data)
            except:
                pass        
        col[i][1].append(data)    
        i+=1

In [9]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [11]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [13]:
df.head()

,Postcode,Borough,Neighbourhood\n
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


#### Rearranging and renaming the columns

In [15]:
df.columns = ['Borough', 'Neighbourhood','Postcode']
cols = df.columns.tolist()
cols
cols = cols[-1:] + cols[:-1]
df = df[cols]
df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned,Not assigned\n,M1A
1,Not assigned,Not assigned\n,M2A
2,North York,Parkwoods\n,M3A
3,North York,Victoria Village\n,M4A
4,Downtown Toronto,Harbourfront\n,M5A


In [16]:
df = df.replace('\n',' ', regex=True)
df.head()

,Postcode,Borough,Neighbourhood
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


In [17]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df = df.reset_index(drop=True)
df.head(10)

,Postcode,Borough,Neighbourhood
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A
8,Queen's Park,Not assigned,M7A
9,Not assigned,Not assigned,M8A


In [18]:
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.columns = ['Postcode','Borough','Neighbourhood']
df.head(10)

,Postcode,Borough,Neighbourhood
0,Central Toronto,Davisville,M4S
1,Central Toronto,Davisville North,M4P
2,Central Toronto,Deer Park,M4V
3,Central Toronto,Forest Hill North,M5P
4,Central Toronto,Forest Hill SE,M4V
5,Central Toronto,Forest Hill West,M5P
6,Central Toronto,Lawrence Park,M4N
7,Central Toronto,Moore Park,M4T
8,Central Toronto,North Midtown,M5R
9,Central Toronto,North Toronto West,M4R


In [19]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()

In [20]:
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [21]:
df[df['Borough'] == 'Queen\'s Park']

,Postcode,Borough,Neighbourhood


In [22]:

df.shape

(211, 3)

In [23]:
df.to_csv(r'df_can.csv')